In [ ]:
import cv2
import os
import shutil
import numpy as np
import zipfile
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Dense, Flatten, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tqdm import tqdm
from PIL import Image, ImageFilter, ImageEnhance
from keras.optimizers import Adam
from keras.regularizers import l2

In [ ]:
# Unzipping the image dataset
zip_ref = zipfile.ZipFile('original.zip', 'r')
zip_ref.extractall('dataset')  # Extracting to the 'dataset' folder
zip_ref.close()

In [ ]:
def apply_mosaic(image, block_size=10):
    width, height = image.size
    mosaic_image = image.copy()

    for i in range(0, width, block_size):
        for j in range(0, height, block_size):
            for k in range(block_size):
                for l in range(block_size):
                    if i + k < width and j + l < height:
                        mosaic_image.putpixel((i + k, j + l), image.getpixel((i, j)))

    return mosaic_image

mosaic_dir = 'dataset/mosaic'
if not os.path.exists(mosaic_dir):
    os.makedirs(mosaic_dir)

# Specify the path to the original images
original_dir = 'dataset/original'
total_images = 20

# Create mosaic images and save
for i in range(1, total_images + 1):
    img_path = f'{original_dir}/{i:02d}.jpg'
    img = Image.open(img_path).convert('RGB')
    for j in range(10):  # Generate 20 mosaic images for each original image
        mosaic_img = apply_mosaic(img, block_size=10)
        mosaic_img.save(f'{mosaic_dir}/mosaic_{i:02d}_{j}.jpg')

In [ ]:
# Define data paths
original_dir = 'dataset/original'
mosaic_dir = 'dataset/mosaic'
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

# Create directories for training and validation
for category in ['original', 'mosaic']:
    os.makedirs(os.path.join(train_dir, category), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, category), exist_ok=True)

# Function to split data and move files
def split_data(source_folder, train_folder, validation_folder, validation_split=0.2):
    files = os.listdir(source_folder)
    train_files, validation_files = train_test_split(files, test_size=validation_split)

    # Copy files to training directory
    for file in train_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(train_folder, file))

    # Copy files to validation directory
    for file in validation_files:
        shutil.copy(os.path.join(source_folder, file), os.path.join(validation_folder, file))

# Apply function to original and distorted datasets
split_data(original_dir, os.path.join(train_dir, 'original'), os.path.join(validation_dir, 'original'))
split_data(mosaic_dir, os.path.join(train_dir, 'mosaic'), os.path.join(validation_dir, 'mosaic'))

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

validation_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
validation_generator = validation_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(224, 224),
    batch_size=5,
    class_mode='categorical'
)

Found 167 images belonging to 2 classes.
Found 63 images belonging to 2 classes.


In [ ]:
import shutil

def remove_unwanted_dirs(directory):
    checkpoint_dir = os.path.join(directory, '.ipynb_checkpoints')
    if os.path.exists(checkpoint_dir):
        shutil.rmtree(checkpoint_dir)
        print(f"Removed {checkpoint_dir}")

remove_unwanted_dirs(train_dir)
remove_unwanted_dirs(validation_dir)

print("Updated training directories:", os.listdir(train_dir))
print("Updated validation directories:", os.listdir(validation_dir))

Updated training directories: ['original', 'mosaic']
Updated validation directories: ['original', 'mosaic']


In [ ]:
dataset_dir = 'dataset'
print("Directories in dataset:", os.listdir(dataset_dir))

for category in os.listdir(dataset_dir):
    category_path = os.path.join(dataset_dir, category)
    if os.path.isdir(category_path):
        print(f"Contents of {category}:", os.listdir(category_path))

Directories in dataset: ['original', 'validation', 'train', 'mosaic']
Contents of original: ['20.jpg', '13.jpg', '06.jpg', '01.jpg', '03.jpg', '18.jpg', '04.jpg', '15.jpg', '16.jpg', '07.jpg', '11.jpg', '09.jpg', '14.jpg', '02.jpg', '19.jpg', '10.jpg', '12.jpg', '08.jpg', '05.jpg', '17.jpg']
Contents of validation: ['original', 'mosaic']
Contents of train: ['original', 'mosaic']
Contents of mosaic: ['mosaic_07_1.jpg', 'mosaic_10_2.jpg', 'mosaic_03_0.jpg', 'mosaic_08_2.jpg', 'mosaic_12_3.jpg', 'mosaic_01_7.jpg', 'mosaic_12_6.jpg', 'mosaic_01_18.jpg', 'mosaic_11_5.jpg', 'mosaic_01_16.jpg', 'mosaic_02_6.jpg', 'mosaic_02_12.jpg', 'mosaic_05_1.jpg', 'mosaic_09_7.jpg', 'mosaic_05_8.jpg', 'mosaic_01_1.jpg', 'mosaic_11_0.jpg', 'mosaic_06_9.jpg', 'mosaic_13_1.jpg', 'mosaic_10_4.jpg', 'mosaic_07_3.jpg', 'mosaic_09_5.jpg', 'mosaic_14_0.jpg', 'mosaic_09_9.jpg', 'mosaic_01_8.jpg', 'mosaic_14_1.jpg', 'mosaic_02_15.jpg', 'mosaic_12_7.jpg', 'mosaic_01_11.jpg', 'mosaic_11_4.jpg', 'mosaic_13_5.jpg', 'mo

In [ ]:
import shutil
import os

directories_to_remove = ['original', 'mosaic']
base_dir = 'dataset'

for dir_name in directories_to_remove:
    dir_path = os.path.join(base_dir, dir_name)
    if os.path.exists(dir_path):
        shutil.rmtree(dir_path)
        print(f"Removed directory: {dir_path}")
    else:
        print(f"Directory does not exist, no need to remove: {dir_path}")

print("Updated contents of the dataset directory:", os.listdir(base_dir))

Removed directory: dataset/original
Removed directory: dataset/mosaic
Updated contents of the dataset directory: ['validation', 'train']


In [ ]:
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
import tensorflow_addons as tfa

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers
x = Flatten()(base_model.output)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)  # L2
x = Dropout(0.5)(x)  # Dropout
predictions = Dense(1, activation='sigmoid')(x)

optimizer = tfa.optimizers.AdamW(learning_rate=0.0001, weight_decay=0.001)

# Compile the model
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Prepare training data
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2  # Use 20% of the data as validation set
)

train_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    'dataset',
    target_size=(224, 224),
    batch_size=5,
    class_mode='binary',
    subset='validation'
)

# Use early stopping to avoid overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Found 185 images belonging to 2 classes.
Found 45 images belonging to 2 classes.


In [ ]:
# @title
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=15,
    epochs=10,
    callbacks=[early_stopping],
    validation_data=validation_generator,
    validation_steps=5
)

# Evaluate the model
results = model.evaluate(validation_generator)
print("Test loss, Test accuracy:", results)

Epoch 1/10
15/15 [==============================] - 64s 4s/step - loss: 5.5094 - accuracy: 0.6267 - val_loss: 8.4392 - val_accuracy: 0.4800
Epoch 2/10
15/15 [==============================] - 64s 4s/step - loss: 5.7389 - accuracy: 0.6667 - val_loss: 11.0826 - val_accuracy: 0.5600
Epoch 3/10
15/15 [==============================] - 57s 4s/step - loss: 7.4064 - accuracy: 0.5600 - val_loss: 7.5021 - val_accuracy: 0.6400
Epoch 4/10
15/15 [==============================] - 65s 4s/step - loss: 4.4807 - accuracy: 0.7200 - val_loss: 5.5019 - val_accuracy: 0.6400
Epoch 5/10
15/15 [==============================] - 62s 4s/step - loss: 3.6913 - accuracy: 0.6800 - val_loss: 7.5573 - val_accuracy: 0.5200
Epoch 6/10
15/15 [==============================] - 66s 4s/step - loss: 4.9170 - accuracy: 0.5733 - val_loss: 12.0251 - val_accuracy: 0.5200
Epoch 7/10
15/15 [==============================] - 61s 4s/step - loss: 5.0167 - accuracy: 0.4667 - val_loss: 8.1230 - val_accuracy: 0.4800
Epoch 8/10
15/15 [